# Master Dataset Construction Notebook

このノートブックは、各interimデータを統合し、  
都道府県×年度粒度の **分析用マスターデータセット** を構築するためのものです。

本Notebookではデータ変換は行わず、  
すでに検証済みのinterim parquetを読み込み、  
キー整合性を確認した上で結合し、masterを生成します。

---

## ■ 目的
- interimデータを再現可能な形で統合する
- 都道府県×年度粒度の分析基盤を確定する
- 列仕様（column_spec）に基づく最終データ構造を作成する

---

## ■ スコープ
本Notebookで行う処理：

- interim parquetの読み込み
- 結合キー整合性チェック
- 左結合による統合
- master保存

行わない処理：

- Rawデータの読み込み
- データクリーニング
- 単位変換
- 指標計算

これらはすべてinterim作成Notebook側で実施済みとする。

---

## ■ 前提条件
- interimデータが `../data/clean/` に存在すること
- 各interimは以下キーを持つこと  
  - `prefecture`
  - `fiscal_year`

---

## ■ 出力
- master parquet  
  `../data/clean/master_dataset.parquet`

---

## ■ 設計原則
- 変換ロジックはinterimに閉じ込める
- masterは統合責務のみ持つ
- 再現性を最優先とする

---

## ■ Last Updated
2026-02-14

In [1]:
from pathlib import Path
import pandas as pd
import re

In [2]:
# ==========================
# Pre-step : Raw Inventory（最初に一度だけ）
# Purpose:
#   - rawフォルダ内のファイル確認
#   - データ源の把握
#   - ファイル名チェック
# Rules:
#   - ETL処理には直接関与しない
#   - 必要なときだけ実行
# ==========================

PROCESSED_DIR = Path("../data/raw")

for ext in ["*.csv", "*.xlsx", "*.pdf"]:
    for p in PROCESSED_DIR.glob(ext):
        print(p.name)

厚労省_医療施設調査_病院数_2024.csv
日本看護協会_夜勤72h超過率_都道府県別_2024.csv
総務省_社会生活統計指標_人口密度_2023.csv
厚労省_衛生行政報告例_看護師数_2023.csv
日本看護協会_給与_都道府県別_2024.csv
日本看護協会_離職率_都道府県別_2023.csv
厚労省_医療施設調査_病床規模_2024.csv
厚労省_賃金構造基本統計調査_看護師年収_2024.xlsx
総務省_社会生活基本調査_通勤時間_2022.xlsx
厚労省_一般職業紹介状況_有効求人倍率_2025.xlsx
総務省_住宅土地統計調査_持ち家率_2024.xlsx
総務省_住宅土地統計調査_家賃_2024.xlsx
日本看護協会_病院看護実態調査_離職率等_2025.pdf


In [3]:
# ==========================
# 0) Config（日本看護協会_離職率_都道府県別_2023.csv）
# ==========================

SUBJECT = "turnover"  # ★CHANGE: "rent_2024" / "jobs" / "night_shift" etc.
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/clean")

# ファイル指定（★CHANGE）
raw_path = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"  # ★CHANGE

# 読み込み方式（★CHANGE：csv / excel）
READ_MODE = "csv"  # ★CHANGE: "csv" or "excel"

# excelの場合（★CHANGE）
EXCEL_SHEET = None  # ★CHANGE: "第１８表ー４　有効求人倍率（実数）" など
EXCEL_HEADER = 0    # ★CHANGE: header行。未確定なら None/0 でプロファイル

# fiscal_year / reference_year（★CHANGE：ルールに従って必ず設定）
FISCAL_YEAR = 2023       # ★CHANGE: master結合キー
REFERENCE_YEAR = 2023    # ★CHANGE: 指標の観測年（年度不一致ならここをズラす）

# 保存先（★CHANGE）
out_path = PROCESSED_DIR / f"interim_{SUBJECT}.parquet"  # ★CHANGE: 命名固定したいならここだけ


In [4]:
# ==========================
# 1-1 Input Check（ファイル存在・文字コードヒント）
# Role:
#   - raw_path の存在確認
#   - 先頭バイト確認（BOM/UTF-16等の検知ヒント）
# Rules:
#   - ここでは pandas 読み込みはしない
# ==========================
assert raw_path.exists(), f"❌ ファイルが存在しません: {raw_path}"
print("target:", raw_path)

with open(raw_path, "rb") as f:
    head = f.read(16)
print("head bytes:", head)


#| エンコーディング | 先頭バイト (16進数) | 特徴・主な用途 |
#| :--- | :--- | :--- |
#| **UTF-8 (BOM付)** | `EF BB BF` | **Excel(Windows)**で作成されたCSVの標準。 |
#| **UTF-16 (LE)** | `FF FE` | Windowsのシステムが出力するUnicode形式。 |
#| **UTF-16 (BE)** | `FE FF` | ビッグエンディアン形式（比較的稀）。 |
#| **BOMなし** | (なし) | **UTF-8 (Naked)** または **Shift-JIS (cp932)**。 |

target: ../data/raw/日本看護協会_離職率_都道府県別_2023.csv
head bytes: b'\xef\xbb\xbfprefecture,tu'


In [5]:
# ==========================
# 1-2 SUBJECT / METRIC定義
# ==========================
# ★CHANGE: SUBJECTを必ず指定
SUBJECT = "turnover"   # 例: "turnover", "rent", "night_shift" など

# ★CHANGE: 対象指標列（英語最終名）
METRIC_COLS = [
    "turnover_total_pct",
    "turnover_new_grad_pct",
    "turnover_experienced_pct",
]

print("\n=== SUBJECT INFO ===")
print("SUBJECT:", SUBJECT)
print("METRIC_COLS:", METRIC_COLS)

# ==========================
# 1-3 Read（Raw → DataFrame）
# Responsibility:
#   - rawファイルをDataFrame化するだけ
#   - 列加工・型変換は禁止（Transformでやる）
# ★CHANGE:
#   READ_MODE / encoding / sheet / header のみ変更可
# ==========================

if READ_MODE == "csv":
    # ★CHANGE: encoding / sep / skiprows 等（データごと）
    df_raw = pd.read_csv(raw_path, encoding="utf-8-sig")
elif READ_MODE == "excel":
    assert EXCEL_SHEET is not None, "❌ excelは EXCEL_SHEET を指定してください"
    df_raw = pd.read_excel(raw_path, sheet_name=EXCEL_SHEET, header=EXCEL_HEADER)
else:
    raise ValueError("❌ READ_MODE must be 'csv' or 'excel'")

print("raw shape:", df_raw.shape)
display(df_raw.head(10))


=== SUBJECT INFO ===
SUBJECT: turnover
METRIC_COLS: ['turnover_total_pct', 'turnover_new_grad_pct', 'turnover_experienced_pct']
raw shape: (47, 4)


,prefecture,turnover_total,turnover_new_grad,turnover_experienced
0,北海道,11.5,5.9,16.6
1,青森県,8.6,10.7,16.7
2,岩手県,6.8,7.8,19.1
3,宮城県,9.1,7.1,12.4
4,秋田県,7.4,5.0,7.3
5,山形県,6.8,6.2,12.7
6,福島県,9.2,7.9,15.4
7,茨城県,10.1,5.4,14.5
8,栃木県,10.2,9.2,16.6
9,群馬県,8.1,9.4,18.2


In [6]:
# ==========================
# 2) Transform（Raw → interim_turnover）
# Responsibility:
#   - 粒度(prefecture × fiscal_year)に整える
#   - 列名を英語最終名へ統一
#   - 型変換・欠損/範囲の前処理（Validationで落とす前提）
# ==========================
import re
import pandas as pd

# --- 0) defensive copy ---
df = df_raw.copy()

# --- 1) prefecture 正規化（冪等） ---
def normalize_prefecture(x: object) -> object:
    if pd.isna(x):
        return x
    s = str(x)

    # 全角/半角スペース除去（何回やっても同じ）
    s = re.sub(r"\s+", "", s)

    # 「全国」「計」などは除外対象（turnoverは既に無い想定だが保険）
    if s in {"全国", "計", "未回答", "無回答・不明", "全国平均"}:
        return None

    # 例外処理（都/府の補完）
    if s == "東京":
        return "東京都"
    if s == "大阪":
        return "大阪府"
    if s == "京都":
        return "京都府"

    # 北海道はそのまま
    if s == "北海道":
        return s

    # 都道府県サフィックスが無ければ補完（すでに付いているなら変えない）
    if not re.search(r"[都道府県]$", s):
        s = s + "県"

    return s

df["prefecture"] = df["prefecture"].apply(normalize_prefecture)

# --- 2) invalid rows drop（キーにならないもの） ---
df = df[df["prefecture"].notna()].copy()

# --- 3) 型変換（数値） ---
# ※ ここで欠損が出たらValidationで落ちる設計
for c in ["turnover_total", "turnover_new_grad", "turnover_experienced"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# --- 4) 英語最終名へ統一（METRIC_COLSに合わせる） ---
rename_map = {
    "turnover_total": "turnover_total_pct",
    "turnover_new_grad": "turnover_new_grad_pct",
    "turnover_experienced": "turnover_experienced_pct",
}
df = df.rename(columns=rename_map)

# --- 5) キー付与（固定値） ---
df["fiscal_year"] = int(FISCAL_YEAR)
df["reference_year"] = int(REFERENCE_YEAR)

# --- 6) 列射影（スキーマ固定） ---
df_interim_turnover = df[["prefecture", "fiscal_year", "reference_year"] + METRIC_COLS].copy()


In [7]:
# ==========================
# 3-1 Validation（保存前に必須：★CHANGEは列リストだけ）
# ==========================
print("\n=== VALIDATION ===")

# ★CHANGE: 指標列（保存対象の数値列）
num_cols = METRIC_COLS

# ★CHANGE: %列（0〜100）
rate_cols = METRIC_COLS        # 例：turnover系はここ
# rate_cols = []               # 例：人口・家賃・病院数などは空

# ★CHANGE: 非負（>=0）
nonneg_cols = []               # 例：人口・家賃・病院数なら num_cols を入れる
# nonneg_cols = METRIC_COLS

df = df_interim_turnover  # ★必要なら df_interim_* に差し替えるだけ

# --- 基本 ---
print("rows:", len(df))
dup = df.duplicated(["prefecture", "fiscal_year"]).sum()
print("dup_keys:", dup)

check_cols = ["prefecture", "fiscal_year", "reference_year"] + num_cols
missing = [c for c in check_cols if c not in df.columns]
assert len(missing) == 0, f"❌ missing columns: {missing}"

nulls = df[check_cols].isna().sum()
print("nulls:\n", nulls)

# --- 範囲チェック（%） ---
def _count_out_of_range(df, cols, lo, hi):
    if not cols:
        return 0
    s = df[cols]
    mask = (s < lo) | (s > hi)
    return int(mask.to_numpy().sum())

# --- 非負チェック ---
def _count_negative(df, cols):
    if not cols:
        return 0
    s = df[cols]
    mask = (s < 0)
    return int(mask.to_numpy().sum())

out_rate = _count_out_of_range(df, rate_cols, 0, 100)
out_neg = _count_negative(df, nonneg_cols)

print("rate out_of_range total:", out_rate)
print("nonneg negative total:", out_neg)

# --- 粒度強制 ---
assert df["fiscal_year"].nunique() == 1
assert df["prefecture"].nunique() == 47

# --- hard asserts ---
assert len(df) == 47
assert dup == 0
assert df[check_cols].isna().sum().sum() == 0
assert out_rate == 0
assert out_neg == 0

print("✅ validation passed")


=== VALIDATION ===
rows: 47
dup_keys: 0
nulls:
 prefecture                  0
fiscal_year                 0
reference_year              0
turnover_total_pct          0
turnover_new_grad_pct       0
turnover_experienced_pct    0
dtype: int64
rate out_of_range total: 0
nonneg negative total: 0
✅ validation passed


In [8]:
# ==========================
# 4-1 Save（interim）
# ==========================
df_interim_turnover.to_parquet(out_path, index=False)
print("✅ saved:", out_path)

# ==========================
# 4-2 Read-back check
# ==========================
df_check = pd.read_parquet(out_path)
print("\n=== READ-BACK ===")
print("shape:", df_check.shape)
print("columns:", df_check.columns.tolist())
display(df_check.head(3))

✅ saved: ../data/clean/interim_turnover.parquet

=== READ-BACK ===
shape: (47, 6)
columns: ['prefecture', 'fiscal_year', 'reference_year', 'turnover_total_pct', 'turnover_new_grad_pct', 'turnover_experienced_pct']


,prefecture,fiscal_year,reference_year,turnover_total_pct,turnover_new_grad_pct,turnover_experienced_pct
0,北海道,2023,2023,11.5,5.9,16.6
1,青森県,2023,2023,8.6,10.7,16.7
2,岩手県,2023,2023,6.8,7.8,19.1


In [9]:
# ==========================
# 0)日本看護協会_夜勤72h超過率_都道府県別_2024.csv
# ==========================
SUBJECT = "night_shift"  # ★CHANGE: "rent_2024" / "jobs" / "night_shift" etc.
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/clean")

# ファイル指定（★CHANGE）
raw_path = RAW_DIR / "日本看護協会_夜勤72h超過率_都道府県別_2024.csv"  # ★CHANGE

# 読み込み方式（★CHANGE：csv / excel）
READ_MODE = "csv"  # ★CHANGE: "csv" or "excel"

# excelの場合（★CHANGE）
EXCEL_SHEET = None  # ★CHANGE: "第１８表ー４　有効求人倍率（実数）" など
EXCEL_HEADER = 0    # ★CHANGE: header行。未確定なら None/0 でプロファイル

# fiscal_year / reference_year（★CHANGE：ルールに従って必ず設定）
FISCAL_YEAR = 2023       # ★CHANGE: master結合キー
REFERENCE_YEAR = 2023    # ★CHANGE: 指標の観測年（年度不一致ならここをズラす）

# 保存先（★CHANGE）
out_path = PROCESSED_DIR / f"interim_{SUBJECT}.parquet"  # ★CHANGE: 命名固定したいならここだけ


In [10]:
# ==========================
# 1-1 Input Check（ファイル存在・文字コードヒント）
# Role:
#   - raw_path の存在確認
#   - 先頭バイト確認（BOM/UTF-16等の検知ヒント）
# Rules:
#   - ここでは pandas 読み込みはしない
# ==========================
assert raw_path.exists(), f"❌ ファイルが存在しません: {raw_path}"
print("target:", raw_path)

with open(raw_path, "rb") as f:
    head = f.read(16)
print("head bytes:", head)

target: ../data/raw/日本看護協会_夜勤72h超過率_都道府県別_2024.csv
head bytes: b'\xef\xbb\xbfprefecture,ni'


In [11]:
# ==========================
# 1-2 SUBJECT / METRIC定義
# ==========================

# ★CHANGE: 対象指標列（英語最終名）
METRIC_COLS = [
   "night_shift_72h_plus_pct",
    "night_shifts_per_month_three_shift",
    "night_shifts_per_month_two_shift",
]

print("\n=== SUBJECT INFO ===")
print("SUBJECT:", SUBJECT)
print("METRIC_COLS:", METRIC_COLS)


# ==========================
# 1-3 Read（Raw → DataFrame）
# Responsibility:
#   - rawファイルをDataFrame化するだけ
#   - 列加工・型変換は禁止（Transformでやる）
# ★CHANGE:
#   READ_MODE / encoding / sheet / header のみ変更可
# ==========================
if READ_MODE == "csv":
    # ★CHANGE: encoding / sep / skiprows 等（データごと）
    df_raw = pd.read_csv(raw_path, encoding="utf-8-sig")
elif READ_MODE == "excel":
    assert EXCEL_SHEET is not None, "❌ excelは EXCEL_SHEET を指定してください"
    df_raw = pd.read_excel(raw_path, sheet_name=EXCEL_SHEET, header=EXCEL_HEADER)
else:
    raise ValueError("❌ READ_MODE must be 'csv' or 'excel'")

print("raw shape:", df_raw.shape)
display(df_raw.head(5))


=== SUBJECT INFO ===
SUBJECT: night_shift
METRIC_COLS: ['night_shift_72h_plus_pct', 'night_shifts_per_month_three_shift', 'night_shifts_per_month_two_shift']
raw shape: (47, 4)


,prefecture,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
0,北海道,36.7,7.8,4.6
1,青森県,36.5,7.7,4.8
2,岩手県,11.8,7.5,4.1
3,宮城県,30.2,8.0,4.7
4,秋田県,25.1,7.7,4.3


In [12]:
# ==========================
# 2) Transform（Raw → interim_night_shift）
# ==========================
# --- defensive copy ---
df = df_raw.copy()

# --- prefecture 正規化（冪等） ---
def normalize_prefecture(x):
    if pd.isna(x):
        return x
    s = str(x)
    s = re.sub(r"\s+", "", s)

    if s in {"全国", "計", "未回答", "無回答・不明", "全国平均"}:
        return None

    if s == "東京":
        return "東京都"
    if s == "大阪":
        return "大阪府"
    if s == "京都":
        return "京都府"
    if s == "北海道":
        return s

    if not re.search(r"[都道府県]$", s):
        s = s + "県"

    return s

df["prefecture"] = df["prefecture"].apply(normalize_prefecture)
df = df[df["prefecture"].notna()].copy()

# --- 数値変換（Validationで最終チェックする前提） ---
raw_metric_cols = [
    "night_shift_72h_plus",
    "night_shifts_per_month_three_shift",
    "night_shifts_per_month_two_shift",
]

for c in raw_metric_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# --- rename（英語最終名へ統一） ---
rename_map = {
    "night_shift_72h_plus": "night_shift_72h_plus_pct",
    "night_shifts_per_month_three_shift": "night_shifts_per_month_three_shift",
    "night_shifts_per_month_two_shift": "night_shifts_per_month_two_shift",
}
df = df.rename(columns=rename_map)

# --- キー付与 ---
df["fiscal_year"] = int(2023)
df["reference_year"] = int(2024)

# --- 列射影（スキーマ固定） ---
df_interim = df[["prefecture", "fiscal_year", "reference_year"] + METRIC_COLS].copy()


In [13]:
# ==========================
# 3-1 Validation（保存前に必須：★CHANGEは列リストだけ）
# ==========================
print("\n=== VALIDATION ===")
num_cols = METRIC_COLS  # ★CHANGE: 数値列（基本はMETRIC_COLS）

# ★CHANGE: %列（0〜100チェック）/ 非負列（>=0チェック）
rate_cols = []     # ★CHANGE: 例 ["turnover_total", ...]
nonneg_cols = []   # ★CHANGE: 例 ["population_total", "家賃平均【円】", ...]

print("rows:", len(df_interim))

dup = df_interim.duplicated(["prefecture", "fiscal_year"]).sum() #
print("dup_keys:", dup)

check_cols = ["prefecture", "fiscal_year", "reference_year"] + num_cols
nulls = df_interim[check_cols].isna().sum()
print("nulls:\n", nulls)

# %の範囲
for c in rate_cols:
    out = ((df_interim[c] < 0) | (df_interim[c] > 100)).sum()
    print(f"{c} out_of_range:", out)

# 非負
for c in nonneg_cols:
    out = (df_interim[c] < 0).sum()
    print(f"{c} negative:", out)

# 粒度強制（47都道府県×年度1つ）
assert df_interim["fiscal_year"].nunique() == 1
assert df_interim["prefecture"].nunique() == 47

# 必須assert
assert len(df_interim) == 47
assert dup == 0
assert df_interim[check_cols].isna().sum().sum() == 0
for c in rate_cols:
    assert ((df_interim[c] < 0) | (df_interim[c] > 100)).sum() == 0
for c in nonneg_cols:
    assert (df_interim[c] < 0).sum() == 0

print("✅ validation passed")



=== VALIDATION ===
rows: 47
dup_keys: 0
nulls:
 prefecture                            0
fiscal_year                           0
reference_year                        0
night_shift_72h_plus_pct              0
night_shifts_per_month_three_shift    0
night_shifts_per_month_two_shift      0
dtype: int64
✅ validation passed


In [14]:
# ==========================
# 4-1 Save（interim）
# ==========================
df_interim.to_parquet(out_path, index=False) #ここ変更必要
print("✅ saved:", out_path)

# ==========================
# 4-2 Read-back check
# ==========================
df_check = pd.read_parquet(out_path)
print("\n=== READ-BACK ===")
print("shape:", df_check.shape)
print("columns:", df_check.columns.tolist())
display(df_check.head(3))

✅ saved: ../data/clean/interim_night_shift.parquet

=== READ-BACK ===
shape: (47, 6)
columns: ['prefecture', 'fiscal_year', 'reference_year', 'night_shift_72h_plus_pct', 'night_shifts_per_month_three_shift', 'night_shifts_per_month_two_shift']


,prefecture,fiscal_year,reference_year,night_shift_72h_plus_pct,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
0,北海道,2023,2024,36.7,7.8,4.6
1,青森県,2023,2024,36.5,7.7,4.8
2,岩手県,2023,2024,11.8,7.5,4.1


In [15]:
# ==========================
# 0) Config（日本看護協会_給与_都道府県別_2024.csv）
# ==========================
SUBJECT = "salary"  # ★CHANGE: "rent_2024" / "jobs" / "night_shift" etc.
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/clean")

# ファイル指定（★CHANGE）
raw_path = RAW_DIR / "日本看護協会_給与_都道府県別_2024.csv"  # ★CHANGE

# 読み込み方式（★CHANGE：csv / excel）
READ_MODE = "csv"  # ★CHANGE: "csv" or "excel"

# excelの場合（★CHANGE）
EXCEL_SHEET = None  # ★CHANGE: "第１８表ー４　有効求人倍率（実数）" など
EXCEL_HEADER = 0    # ★CHANGE: header行。未確定なら None/0 でプロファイル

# fiscal_year / reference_year（★CHANGE：ルールに従って必ず設定）
FISCAL_YEAR = 2023       # ★CHANGE: master結合キー
REFERENCE_YEAR = 2023    # ★CHANGE: 指標の観測年（年度不一致ならここをズラす）

# 保存先（★CHANGE）
out_path = PROCESSED_DIR / f"interim_{SUBJECT}.parquet"  # ★CHANGE: 命名固定したいならここだけ


In [16]:
# ==========================
# 1-1 Input Check（ファイル存在・文字コードヒント）
# Role:
#   - raw_path の存在確認
#   - 先頭バイト確認（BOM/UTF-16等の検知ヒント）
# Rules:
#   - ここでは pandas 読み込みはしない
# ==========================
assert raw_path.exists(), f"❌ ファイルが存在しません: {raw_path}"
print("target:", raw_path)

with open(raw_path, "rb") as f:
    head = f.read(16)
print("head bytes:", head)

target: ../data/raw/日本看護協会_給与_都道府県別_2024.csv
head bytes: b'\xef\xbb\xbfprefecture,st'


In [17]:
# ==========================
# 1-2 SUBJECT / METRIC定義
# ==========================

# ★CHANGE: 対象指標列（英語最終名）
METRIC_COLS = [
    "starting_salary_nurse_diploma_monthly_yen",
    "starting_salary_nurse_bachelor_monthly_yen",
    "salary_nurse_10yr_non_manager_monthly_yen",
]

print("\n=== SUBJECT INFO ===")
print("SUBJECT:", SUBJECT)
print("METRIC_COLS:", METRIC_COLS)

# ==========================
# 1-3 Read（Raw → DataFrame）
# Responsibility:
#   - rawファイルをDataFrame化するだけ
#   - 列加工・型変換は禁止（Transformでやる）
# ★CHANGE:
#   READ_MODE / encoding / sheet / header のみ変更可
# ==========================
if READ_MODE == "csv":
    # ★CHANGE: encoding / sep / skiprows 等（データごと）
    df_raw = pd.read_csv(raw_path, encoding="utf-8-sig")
elif READ_MODE == "excel":
    assert EXCEL_SHEET is not None, "❌ excelは EXCEL_SHEET を指定してください"
    df_raw = pd.read_excel(raw_path, sheet_name=EXCEL_SHEET, header=EXCEL_HEADER)
else:
    raise ValueError("❌ READ_MODE must be 'csv' or 'excel'")

print("raw shape:", df_raw.shape)
display(df_raw.head(5))


=== SUBJECT INFO ===
SUBJECT: salary
METRIC_COLS: ['starting_salary_nurse_diploma_monthly_yen', 'starting_salary_nurse_bachelor_monthly_yen', 'salary_nurse_10yr_non_manager_monthly_yen']
raw shape: (47, 4)


,prefecture,starting_salary_nurse_diploma_monthly_yen,starting_salary_nurse_bachelor_monthly_yen,salary_nurse_10yr_non_manager_monthly_yen
0,北海道,270231.0,276652.0,326530.0
1,青森県,270494.0,275476.0,314135.0
2,岩手県,268264.0,276926.0,322466.0
3,宮城県,272385.0,280377.0,332670.0
4,秋田県,263109.0,272553.0,321717.0


In [18]:
# ==========================
# 2) Transform（Raw → interim）
# ==========================
import re
import pandas as pd

df = df_raw.copy()

# --- prefecture 正規化（冪等） ---
def normalize_prefecture(x):
    if pd.isna(x):
        return x
    s = str(x)
    s = re.sub(r"\s+", "", s)

    if s in {"全国", "計", "未回答", "無回答・不明", "全国平均"}:
        return None

    if s == "東京":
        return "東京都"
    if s == "大阪":
        return "大阪府"
    if s == "京都":
        return "京都府"
    if s == "北海道":
        return s

    if not re.search(r"[都道府県]$", s):
        s = s + "県"

    return s

df["prefecture"] = df["prefecture"].apply(normalize_prefecture)
df = df[df["prefecture"].notna()].copy()

# --- 数値変換（salaryは非負チェック用。NULLはValidationで落とす） ---
for c in METRIC_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# --- キー付与 ---
df["fiscal_year"] = int(FISCAL_YEAR)
df["reference_year"] = int(REFERENCE_YEAR)

# --- スキーマ固定 ---
df_interim = df[["prefecture", "fiscal_year", "reference_year"] + METRIC_COLS].copy()

print("interim columns:", df_interim.columns.tolist())


interim columns: ['prefecture', 'fiscal_year', 'reference_year', 'starting_salary_nurse_diploma_monthly_yen', 'starting_salary_nurse_bachelor_monthly_yen', 'salary_nurse_10yr_non_manager_monthly_yen']


In [19]:
# ==========================
# 3-1 Validation（保存前に必須：★CHANGEは列リストだけ）
# ==========================
print("\n=== VALIDATION ===")
num_cols = METRIC_COLS  # ★CHANGE: 数値列（基本はMETRIC_COLS）

# ★CHANGE: %列（0〜100チェック）/ 非負列（>=0チェック）
rate_cols = []     # ★CHANGE: 例 ["turnover_total", ...]
nonneg_cols = []   # ★CHANGE: 例 ["population_total", "家賃平均【円】", ...]

print("rows:", len(df_interim))

dup = df_interim.duplicated(["prefecture", "fiscal_year"]).sum() #
print("dup_keys:", dup)

check_cols = ["prefecture", "fiscal_year", "reference_year"] + num_cols
nulls = df_interim[check_cols].isna().sum()
print("nulls:\n", nulls)

# %の範囲
for c in rate_cols:
    out = ((df_interim[c] < 0) | (df_interim[c] > 100)).sum()
    print(f"{c} out_of_range:", out)

# 非負
for c in nonneg_cols:
    out = (df_interim[c] < 0).sum()
    print(f"{c} negative:", out)

# 粒度強制（47都道府県×年度1つ）
assert df_interim["fiscal_year"].nunique() == 1
assert df_interim["prefecture"].nunique() == 47

# 必須assert
assert len(df_interim) == 47
assert dup == 0
assert df_interim[check_cols].isna().sum().sum() == 0
for c in rate_cols:
    assert ((df_interim[c] < 0) | (df_interim[c] > 100)).sum() == 0
for c in nonneg_cols:
    assert (df_interim[c] < 0).sum() == 0

print("✅ validation passed")



=== VALIDATION ===
rows: 47
dup_keys: 0
nulls:
 prefecture                                    0
fiscal_year                                   0
reference_year                                0
starting_salary_nurse_diploma_monthly_yen     0
starting_salary_nurse_bachelor_monthly_yen    0
salary_nurse_10yr_non_manager_monthly_yen     0
dtype: int64
✅ validation passed


In [20]:
# ==========================
# 4-1 Save（interim）
# ==========================
df_interim.to_parquet(out_path, index=False) #ここ変更必要
print("✅ saved:", out_path)

# ==========================
# 4-2 Read-back check
# ==========================
df_check = pd.read_parquet(out_path)
print("\n=== READ-BACK ===")
print("shape:", df_check.shape)
print("columns:", df_check.columns.tolist())
display(df_check.head(3))

✅ saved: ../data/clean/interim_salary.parquet

=== READ-BACK ===
shape: (47, 6)
columns: ['prefecture', 'fiscal_year', 'reference_year', 'starting_salary_nurse_diploma_monthly_yen', 'starting_salary_nurse_bachelor_monthly_yen', 'salary_nurse_10yr_non_manager_monthly_yen']


,prefecture,fiscal_year,reference_year,starting_salary_nurse_diploma_monthly_yen,starting_salary_nurse_bachelor_monthly_yen,salary_nurse_10yr_non_manager_monthly_yen
0,北海道,2023,2023,270231.0,276652.0,326530.0
1,青森県,2023,2023,270494.0,275476.0,314135.0
2,岩手県,2023,2023,268264.0,276926.0,322466.0


In [21]:
# ==========================
# 0) Config（厚労省_賃金構造基本統計調査_看護師年収_2024.xlsx）
# ==========================
SUBJECT = "nurse_wage"
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/clean")

raw_path = RAW_DIR / "厚労省_賃金構造基本統計調査_看護師年収_2024.xlsx"

READ_MODE = "excel"

EXCEL_SHEET = 0
EXCEL_HEADER = None   # ←まずはプロファイル

FISCAL_YEAR = 2023
REFERENCE_YEAR = 2023

out_path = PROCESSED_DIR / f"interim_{SUBJECT}.parquet"


In [22]:
# ==========================
# 1-1 Input Check（ファイル存在・文字コードヒント）
# Role:
#   - raw_path の存在確認
#   - 先頭バイト確認（BOM/UTF-16等の検知ヒント）
# Rules:
#   - ここでは pandas 読み込みはしない
# ==========================
assert raw_path.exists(), f"❌ ファイルが存在しません: {raw_path}"
print("target:", raw_path)

with open(raw_path, "rb") as f:
    head = f.read(16)
print("head bytes:", head)

target: ../data/raw/厚労省_賃金構造基本統計調査_看護師年収_2024.xlsx
head bytes: b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00b\xee'


In [23]:
# ==========================
# 1-2 SUBJECT / METRIC定義
# ==========================

# ★CHANGE: 対象指標列（英語最終名）
METRIC_COLS = [
    "nurse_avg_age_years",
    "nurse_avg_tenure_years",
    "nurse_overtime_actual_hours",
    "nurse_cash_earnings_fixed_monthly_yen",
    "nurse_bonus_special_annual_yen",
]

print("\n=== SUBJECT INFO ===")
print("SUBJECT:", SUBJECT)
print("METRIC_COLS:", METRIC_COLS)


# ==========================
# 1-3 Read（Raw → DataFrame）
# Responsibility:
#   - rawファイルをDataFrame化するだけ
#   - 列加工・型変換は禁止（Transformでやる）
# ★CHANGE:
#   READ_MODE / encoding / sheet / header のみ変更可
# ==========================
if READ_MODE == "csv":
    # ★CHANGE: encoding / sep / skiprows 等（データごと）
    df_raw = pd.read_csv(raw_path, encoding="utf-8-sig")
elif READ_MODE == "excel":
    assert EXCEL_SHEET is not None
    df_raw = pd.read_excel(raw_path, sheet_name=EXCEL_SHEET, header=12)
else:
    raise ValueError("❌ READ_MODE must be 'csv' or 'excel'")

print("raw shape:", df_raw.shape)
display(df_raw.head(5))


=== SUBJECT INFO ===
SUBJECT: nurse_wage
METRIC_COLS: ['nurse_avg_age_years', 'nurse_avg_tenure_years', 'nurse_overtime_actual_hours', 'nurse_cash_earnings_fixed_monthly_yen', 'nurse_bonus_special_annual_yen']
raw shape: (48, 29)


,性別_基本 コード,性別_基本 補助コード,性別_基本,職種（小分類）（2020～） コード,職種（小分類）（2020～） 補助コード,職種（小分類）（2020～）,時間軸（2020～2023） コード,時間軸（2020～2023） 補助コード,時間軸（2020～2023）,地域 コード,...,超過実労働時間数【時間】,注釈.3,きまって支給する現金給与額【千円】,注釈.4,所定内給与額【千円】,注釈.5,年間賞与その他特別給与額【千円】,注釈.6,労働者数【十人】,注釈.7
0,1,NaN,男女計,1133,NaN,看護師,2023000000,NaN,2023年,0,...,6,NaN,352.1,NaN,319.3,NaN,856.5,NaN,83501,NaN
1,1,NaN,男女計,1133,NaN,看護師,2023000000,NaN,2023年,1000,...,5,NaN,334.0,NaN,311.4,NaN,781.0,NaN,4587,NaN
2,1,NaN,男女計,1133,NaN,看護師,2023000000,NaN,2023年,2000,...,5,NaN,301.6,NaN,274.3,NaN,733.0,NaN,821,NaN
3,1,NaN,男女計,1133,NaN,看護師,2023000000,NaN,2023年,3000,...,3,NaN,310.6,NaN,298.3,NaN,862.5,NaN,803,NaN
4,1,NaN,男女計,1133,NaN,看護師,2023000000,NaN,2023年,4000,...,6,NaN,365.8,NaN,320.0,NaN,968.4,NaN,1624,NaN


In [24]:
# ==========================
# 2) Transform（Raw → interim）
# ==========================
df = df_raw.copy()

df = df.rename(columns={"地域": "prefecture"})
df["prefecture"] = df["prefecture"].apply(normalize_prefecture)
df = df.dropna(subset=["prefecture"]).copy()

df["fiscal_year"] = FISCAL_YEAR
df["reference_year"] = REFERENCE_YEAR

rename_map = {
    "年齢【歳】": "nurse_avg_age_years",
    "勤続年数【年】": "nurse_avg_tenure_years",
    "超過実労働時間数【時間】": "nurse_overtime_actual_hours",
    "きまって支給する現金給与額【千円】": "nurse_cash_earnings_fixed_monthly_yen",
    "年間賞与その他特別給与額【千円】": "nurse_bonus_special_annual_yen",
}
df = df.rename(columns=rename_map)

for c in METRIC_COLS:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df_interim = df[["prefecture", "fiscal_year", "reference_year"] + METRIC_COLS].copy()


In [25]:
# ==========================
# 3-1 Validation（保存前に必須：★CHANGEは列リストだけ）
# ==========================
print("\n=== VALIDATION ===")
num_cols = METRIC_COLS  # ★CHANGE: 数値列（基本はMETRIC_COLS）

# ★CHANGE: %列（0〜100チェック）/ 非負列（>=0チェック）
rate_cols = []     # ★CHANGE: 例 ["turnover_total", ...]
nonneg_cols = []   # ★CHANGE: 例 ["population_total", "家賃平均【円】", ...]

print("rows:", len(df_interim))

dup = df_interim.duplicated(["prefecture", "fiscal_year"]).sum() #
print("dup_keys:", dup)

check_cols = ["prefecture", "fiscal_year", "reference_year"] + num_cols
nulls = df_interim[check_cols].isna().sum()
print("nulls:\n", nulls)

# %の範囲
for c in rate_cols:
    out = ((df_interim[c] < 0) | (df_interim[c] > 100)).sum()
    print(f"{c} out_of_range:", out)

# 非負
for c in nonneg_cols:
    out = (df_interim[c] < 0).sum()
    print(f"{c} negative:", out)

# 粒度強制（47都道府県×年度1つ）
assert df_interim["fiscal_year"].nunique() == 1
assert df_interim["prefecture"].nunique() == 47

# 必須assert
assert len(df_interim) == 47
assert dup == 0
assert df_interim[check_cols].isna().sum().sum() == 0
for c in rate_cols:
    assert ((df_interim[c] < 0) | (df_interim[c] > 100)).sum() == 0
for c in nonneg_cols:
    assert (df_interim[c] < 0).sum() == 0

print("✅ validation passed")


=== VALIDATION ===
rows: 47
dup_keys: 0
nulls:
 prefecture                               0
fiscal_year                              0
reference_year                           0
nurse_avg_age_years                      0
nurse_avg_tenure_years                   0
nurse_overtime_actual_hours              0
nurse_cash_earnings_fixed_monthly_yen    0
nurse_bonus_special_annual_yen           0
dtype: int64
✅ validation passed


In [26]:
# ==========================
# 4-1 Save（interim）
# ==========================
df_interim.to_parquet(out_path, index=False) #ここ変更必要
print("✅ saved:", out_path)

# ==========================
# 4-2 Read-back check
# ==========================
df_check = pd.read_parquet(out_path)
print("\n=== READ-BACK ===")
print("shape:", df_check.shape)
print("columns:", df_check.columns.tolist())
display(df_check.head(3))

✅ saved: ../data/clean/interim_nurse_wage.parquet

=== READ-BACK ===
shape: (47, 8)
columns: ['prefecture', 'fiscal_year', 'reference_year', 'nurse_avg_age_years', 'nurse_avg_tenure_years', 'nurse_overtime_actual_hours', 'nurse_cash_earnings_fixed_monthly_yen', 'nurse_bonus_special_annual_yen']


,prefecture,fiscal_year,reference_year,nurse_avg_age_years,nurse_avg_tenure_years,nurse_overtime_actual_hours,nurse_cash_earnings_fixed_monthly_yen,nurse_bonus_special_annual_yen
0,北海道,2023,2023,42.0,8.0,5,334.0,781.0
1,青森県,2023,2023,42.0,11.9,5,301.6,733.0
2,岩手県,2023,2023,46.0,11.8,3,310.6,862.5


In [27]:
# ==========================
# 0) Config（日本看護協会_離職率_都道府県別_2023.csv）
# ==========================
SUBJECT = ""  # ★CHANGE: "rent_2024" / "jobs" / "night_shift" etc.
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/clean")

# ファイル指定（★CHANGE）
raw_path = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"  # ★CHANGE

# 読み込み方式（★CHANGE：csv / excel）
READ_MODE = "csv"  # ★CHANGE: "csv" or "excel"

# excelの場合（★CHANGE）
EXCEL_SHEET = None  # ★CHANGE: "第１８表ー４　有効求人倍率（実数）" など
EXCEL_HEADER = 0    # ★CHANGE: header行。未確定なら None/0 でプロファイル

# fiscal_year / reference_year（★CHANGE：ルールに従って必ず設定）
FISCAL_YEAR = 2023       # ★CHANGE: master結合キー
REFERENCE_YEAR = 2023    # ★CHANGE: 指標の観測年（年度不一致ならここをズラす）

# 保存先（★CHANGE）
out_path = PROCESSED_DIR / f"interim_{SUBJECT}.parquet"  # ★CHANGE: 命名固定したいならここだけ

In [28]:
# ==========================
# 0) Config（総務省_社会生活統計指標_人口密度_2023.csv）
# ==========================
SUBJECT = "population"  # ★CHANGE: "rent_2024" / "jobs" / "night_shift" etc.
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/clean")

# ファイル指定（★CHANGE）
raw_path = RAW_DIR / "総務省_社会生活統計指標_人口密度_2023.csv"  # ★CHANGE

# 読み込み方式（★CHANGE：csv / excel）
READ_MODE = "csv"  # ★CHANGE: "csv" or "excel"

# excelの場合（★CHANGE）
EXCEL_SHEET = None  # ★CHANGE: "第１８表ー４　有効求人倍率（実数）" など
EXCEL_HEADER = 0    # ★CHANGE: header行。未確定なら None/0 でプロファイル

# fiscal_year / reference_year（★CHANGE：ルールに従って必ず設定）
FISCAL_YEAR = 2023       # ★CHANGE: master結合キー
REFERENCE_YEAR = 2023    # ★CHANGE: 指標の観測年（年度不一致ならここをズラす）

# 保存先（★CHANGE）
out_path = PROCESSED_DIR / f"interim_{SUBJECT}.parquet"  # ★CHANGE: 命名固定したいならここだけ

In [29]:
# ==========================
# 1-1 Input Check（ファイル存在・文字コードヒント）
# Role:
#   - raw_path の存在確認
#   - 先頭バイト確認（BOM/UTF-16等の検知ヒント）
# Rules:
#   - ここでは pandas 読み込みはしない
# ==========================
assert raw_path.exists(), f"❌ ファイルが存在しません: {raw_path}"
print("target:", raw_path)

with open(raw_path, "rb") as f:
    head = f.read(16)
print("head bytes:", head)


target: ../data/raw/総務省_社会生活統計指標_人口密度_2023.csv
head bytes: b'\xef\xbb\xbf\xe7\xb5\xb1\xe8\xa8\x88\xe5\x90\x8d\xef\xbc\x9a,'


In [30]:
# ==========================
# 1-2 PROFILE READ（ヘッダー探索用）
# ==========================
df_test = pd.read_csv(raw_path, header=None, encoding="utf-8-sig")
display(df_test.head(20))

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,統計名：,都道府県データ 社会生活統計指標,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,表番号：,10201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,表題：,Ａ 人口・世帯,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,実施年月：,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,市区町村時点（年月日）：,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,***,調査又は集計していないもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-,データが得られないもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,X,数値が秘匿されているもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# ==========================
# 1-3 SUBJECT / METRIC定義
# ==========================

# ★CHANGE: 対象指標列（英語最終名）
METRIC_COLS = [
    "population_total", "population_density_per_km2"
]

print("\n=== SUBJECT INFO ===")
print("SUBJECT:", SUBJECT)
print("METRIC_COLS:", METRIC_COLS)


# ==========================
# 1-4 Read（Raw → DataFrame）
# Responsibility:
#   - rawファイルをDataFrame化するだけ
#   - 列加工・型変換は禁止（Transformでやる）
# ★CHANGE:
#   READ_MODE / encoding / sheet / header のみ変更可
# ==========================
if READ_MODE == "csv":
    # ★CHANGE: encoding / sep / skiprows 等（データごと）
    df_raw = pd.read_csv(raw_path, header=12, encoding="utf-8-sig")
elif READ_MODE == "excel":
    assert EXCEL_SHEET is not None, "❌ excelは EXCEL_SHEET を指定してください"
    df_raw = pd.read_excel(raw_path, sheet_name=EXCEL_SHEET, header=EXCEL_HEADER)
else:
    raise ValueError("❌ READ_MODE must be 'csv' or 'excel'")

print("\n=== RAW COLUMNS LIST ===")
print(list(df_raw.columns))

print("raw shape:", df_raw.shape)
display(df_raw.head(5))


=== SUBJECT INFO ===
SUBJECT: population
METRIC_COLS: ['population_total', 'population_density_per_km2']

=== RAW COLUMNS LIST ===
['調査年 コード', '調査年 補助コード', '調査年', '地域 コード', '地域 補助コード', '地域', '/Ａ\u3000人口・世帯', '#A011000_総人口【万人】', '#A0110001_総人口（男）【万人】', '#A0110002_総人口（女）【万人】', '#A01101_全国総人口に占める人口割合（A1101/A1101(全国)）【％】', '#A01201_総面積１km2当たり人口密度【人】', '#A01202_可住地面積１km2当たり人口密度【人】', '#A0191002_将来推計人口（2025年）【人】', '#A0191003_将来推計人口（2030年）【人】', '#A0191004_将来推計人口（2035年）【人】', '#A0191005_将来推計人口（2040年）【人】', '#A0191006_将来推計人口（2045年）【人】', '#A0191007_将来推計人口（2050年）【人】', '#A02101_人口性比（総数）（A110101/A110102）【‐】', '#A02102_人口性比（15歳未満人口）(A130101/A130102)【‐】', '#A02103_人口性比（15～64歳人口）(A130201/A130202)【‐】', '#A02104_人口性比（65歳以上人口) (A130301/A130302)【‐】', '#A03501_15歳未満人口割合【％】', '#A03502_15～64歳人口割合【％】', '#A03503_65歳以上人口割合【％】', '#A05101_人口増減率（（A1101/A1101（-1））-1）【％】', '#A05301_転入超過率（日本人移動者）【％】', '#A05302_転入率（日本人移動者）【％】', '#A05303_転出率（日本人移動者）【％】', '#A05307_転入超過率【％】', '#A05308_転入率【％】', '#A05309_転出率【％】']
raw shape: 

,調査年 コード,調査年 補助コード,調査年,地域 コード,地域 補助コード,地域,/Ａ 人口・世帯,#A011000_総人口【万人】,#A0110001_総人口（男）【万人】,#A0110002_総人口（女）【万人】,...,#A03501_15歳未満人口割合【％】,#A03502_15～64歳人口割合【％】,#A03503_65歳以上人口割合【％】,#A05101_人口増減率（（A1101/A1101（-1））-1）【％】,#A05301_転入超過率（日本人移動者）【％】,#A05302_転入率（日本人移動者）【％】,#A05303_転出率（日本人移動者）【％】,#A05307_転入超過率【％】,#A05308_転入率【％】,#A05309_転出率【％】
0,2023100000,NaN,2023年度,0,NaN,全国,NaN,"12,435","6,049","6,386",...,11.4,59.5,29.1,-0.48,-,1.79,1.79,-,2.05,2.05
1,2023100000,NaN,2023年度,1000,NaN,北海道,NaN,509,241,269,...,10.1,56.9,33.0,-0.93,-0.11,0.93,1.04,-0.1,1.05,1.15
2,2023100000,NaN,2023年度,2000,NaN,青森県,NaN,118,56,63,...,10.0,54.8,35.2,-1.66,-0.47,1.29,1.76,-0.48,1.37,1.85
3,2023100000,NaN,2023年度,3000,NaN,岩手県,NaN,116,56,60,...,10.3,54.7,35.0,-1.52,-0.41,1.28,1.69,-0.4,1.39,1.79
4,2023100000,NaN,2023年度,4000,NaN,宮城県,NaN,226,111,116,...,11.1,59.7,29.2,-0.70,-0.04,1.90,1.94,-0.06,2.03,2.09


In [32]:
# ==========================
# 2) Transform（Raw → interim）
# ==========================
# NOTE: populationは列が明確に特定できるため固定抽出

df = df_raw.copy()

# prefecture
if "地域" not in df.columns:
    raise KeyError('Required column not found: "地域"')
df["prefecture"] = df["地域"].apply(normalize_prefecture)

# 全国など除外
df = df[df["prefecture"].notna()].copy()

# year（SUBJECT INFOで必須）
df["fiscal_year"] = int(FISCAL_YEAR)
df["reference_year"] = int(REFERENCE_YEAR)

# --- raw列を固定指定で抽出 ---
raw_total = "#A011000_総人口【万人】"
raw_density = "#A01201_総面積１km2当たり人口密度【人】"

for col in [raw_total, raw_density]:
    if col not in df.columns:
        raise KeyError(f"Missing raw column: {col}")

df = df.rename(
    columns={
        raw_total: "population_total",
        raw_density: "population_density_per_km2",
    }
)

# 数値化（補完なし）
for col in METRIC_COLS:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .replace({"-": None, "—": None, "–": None})
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

# schema固定
df_interim = df[["prefecture", "fiscal_year", "reference_year"] + METRIC_COLS].copy()


In [33]:
# ==========================
# 3-1 Validation（保存前に必須：★CHANGEは列リストだけ）
# ==========================
print("\n=== VALIDATION ===")
num_cols = METRIC_COLS  # ★CHANGE: 数値列（基本はMETRIC_COLS）

# ★CHANGE: %列（0〜100チェック）/ 非負列（>=0チェック）
rate_cols = []     # ★CHANGE: 例 ["turnover_total", ...]
nonneg_cols = []   # ★CHANGE: 例 ["population_total", "家賃平均【円】", ...]

print("rows:", len(df_interim))

dup = df_interim.duplicated(["prefecture", "fiscal_year"]).sum() #
print("dup_keys:", dup)

check_cols = ["prefecture", "fiscal_year", "reference_year"] + num_cols
nulls = df_interim[check_cols].isna().sum()
print("nulls:\n", nulls)

# %の範囲
for c in rate_cols:
    out = ((df_interim[c] < 0) | (df_interim[c] > 100)).sum()
    print(f"{c} out_of_range:", out)

# 非負
for c in nonneg_cols:
    out = (df_interim[c] < 0).sum()
    print(f"{c} negative:", out)

# 粒度強制（47都道府県×年度1つ）
assert df_interim["fiscal_year"].nunique() == 1
assert df_interim["prefecture"].nunique() == 47

# 必須assert
assert len(df_interim) == 47
assert dup == 0
assert df_interim[check_cols].isna().sum().sum() == 0
for c in rate_cols:
    assert ((df_interim[c] < 0) | (df_interim[c] > 100)).sum() == 0
for c in nonneg_cols:
    assert (df_interim[c] < 0).sum() == 0

print("✅ validation passed")



=== VALIDATION ===
rows: 47
dup_keys: 0
nulls:
 prefecture                    0
fiscal_year                   0
reference_year                0
population_total              0
population_density_per_km2    0
dtype: int64
✅ validation passed


In [34]:
# ==========================
# 4-1 Save（interim）
# ==========================
df_interim.to_parquet(out_path, index=False) #ここ変更必要
print("✅ saved:", out_path)

# ==========================
# 4-2 Read-back check
# ==========================
df_check = pd.read_parquet(out_path)
print("\n=== READ-BACK ===")
print("shape:", df_check.shape)
print("columns:", df_check.columns.tolist())
display(df_check.head(3))

✅ saved: ../data/clean/interim_population.parquet

=== READ-BACK ===
shape: (47, 5)
columns: ['prefecture', 'fiscal_year', 'reference_year', 'population_total', 'population_density_per_km2']


,prefecture,fiscal_year,reference_year,population_total,population_density_per_km2
0,北海道,2023,2023,509,64.9
1,青森県,2023,2023,118,122.8
2,岩手県,2023,2023,116,76.1


In [35]:

# ==========================
# 0) Config（厚労省_医療施設調査_病院数_2024.csv）
# ==========================
SUBJECT = ""  # ★CHANGE: "rent_2024" / "jobs" / "night_shift" etc.
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/clean")

# ファイル指定（★CHANGE）
raw_path = RAW_DIR / "厚労省_医療施設調査_病院数_2024.csv"  # ★CHANGE

# 読み込み方式（★CHANGE：csv / excel）
READ_MODE = "csv"  # ★CHANGE: "csv" or "excel"

# excelの場合（★CHANGE）
EXCEL_SHEET = None  # ★CHANGE: "第１８表ー４　有効求人倍率（実数）" など
EXCEL_HEADER = 0    # ★CHANGE: header行。未確定なら None/0 でプロファイル

# fiscal_year / reference_year（★CHANGE：ルールに従って必ず設定）
FISCAL_YEAR = 2023       # ★CHANGE: master結合キー
REFERENCE_YEAR = 2023    # ★CHANGE: 指標の観測年（年度不一致ならここをズラす）

# 保存先（★CHANGE）
out_path = PROCESSED_DIR / f"interim_{SUBJECT}.parquet"  # ★CHANGE: 命名固定したいならここだけ


In [36]:
# ==========================
# 1-1 Input Check（ファイル存在・文字コードヒント）
# Role:
#   - raw_path の存在確認
#   - 先頭バイト確認（BOM/UTF-16等の検知ヒント）
# Rules:
#   - ここでは pandas 読み込みはしない
# ==========================
assert raw_path.exists(), f"❌ ファイルが存在しません: {raw_path}"
print("target:", raw_path)

with open(raw_path, "rb") as f:
    head = f.read(16)
print("head bytes:", head)


target: ../data/raw/厚労省_医療施設調査_病院数_2024.csv
head bytes: b'\xef\xbb\xbf\xe7\xb5\xb1\xe8\xa8\x88\xe5\x90\x8d\xef\xbc\x9a,'


In [37]:
# ==========================
# 1-2 PROFILE READ（ヘッダー探索用）
# ==========================
df_test = pd.read_csv(raw_path, header=None, encoding="utf-8-sig")
display(df_test.head(20))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,統計名：,医療施設調査 令和５年医療施設（静態・動態）調査 都道府県編,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,表番号：,T1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,表題：,第１表 病院数，年次・都道府県別,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,実施年月：,2023年,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,***,数字が得られないもの,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-,計数のない場合,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,…,…,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,,E0020のCSV内の空欄,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# ==========================
# 1-3 SUBJECT / METRIC定義
# ==========================

# ★CHANGE: 対象指標列（英語最終名）
METRIC_COLS = ["hospitals_count", "hospitals_per_100k"]

print("\n=== SUBJECT INFO ===")
print("SUBJECT:", SUBJECT)
print("METRIC_COLS:", METRIC_COLS)


# ==========================
# 1-3 Read（Raw → DataFrame）
# Responsibility:
#   - rawファイルをDataFrame化するだけ
#   - 列加工・型変換は禁止（Transformでやる）
# ★CHANGE:
#   READ_MODE / encoding / sheet / header のみ変更可
# ==========================
if READ_MODE == "csv":
    # ★CHANGE: encoding / sep / skiprows 等（データごと）
    df_raw = pd.read_csv(raw_path, header=14, encoding="utf-8-sig")
elif READ_MODE == "excel":
    assert EXCEL_SHEET is not None, "❌ excelは EXCEL_SHEET を指定してください"
    df_raw = pd.read_excel(raw_path, sheet_name=EXCEL_SHEET, header=EXCEL_HEADER)
else:
    raise ValueError("❌ READ_MODE must be 'csv' or 'excel'")

print("\n=== RAW INFO ===")
print("shape:", df_raw.shape)
print("columns:", list(df_raw.columns))
print("dtypes:\n", df_raw.dtypes)
display(df_raw.head(5))


=== SUBJECT INFO ===
SUBJECT: 
METRIC_COLS: ['hospitals_count', 'hospitals_per_100k']

=== RAW INFO ===
shape: (48, 20)
columns: ['表章項目 コード', '表章項目 補助コード', '表章項目', '調査年10 コード', '調査年10 補助コード', '調査年10', '都道府県_005 コード', '都道府県_005 補助コード', '都道府県_005', '/年次_043(人口10万対）', '平成14年(2002年)', '平成17年(2005年)', '平成20年(2008年)', '平成23年(2011年)', '平成26年(2014年)', '平成29年(2017年)', '令和2年(2020年)', '令和4年(2022年)', '令和5年(2023年)（実数）', '令和5年(2023年)（人口10万対）']
dtypes:
 表章項目 コード                 int64
表章項目 補助コード             float64
表章項目                    object
調査年10 コード                int64
調査年10 補助コード            float64
調査年10                   object
都道府県_005 コード             int64
都道府県_005 補助コード         float64
都道府県_005                object
/年次_043(人口10万対）        float64
平成14年(2002年)            object
平成17年(2005年)            object
平成20年(2008年)            object
平成23年(2011年)            object
平成26年(2014年)            object
平成29年(2017年)            object
令和2年(2020年)             object
令和4年(2022年)             objec

,表章項目 コード,表章項目 補助コード,表章項目,調査年10 コード,調査年10 補助コード,調査年10,都道府県_005 コード,都道府県_005 補助コード,都道府県_005,/年次_043(人口10万対）,平成14年(2002年),平成17年(2005年),平成20年(2008年),平成23年(2011年),平成26年(2014年),平成29年(2017年),令和2年(2020年),令和4年(2022年),令和5年(2023年)（実数）,令和5年(2023年)（人口10万対）
0,50,NaN,病院数,2023000000,NaN,令和5年(2023年),1,NaN,全 国,NaN,"9,187","9,026","8,794","8,605","8,493","8,412","8,238","8,156","8,122",6.5
1,50,NaN,病院数,2023000000,NaN,令和5年(2023年),2,NaN,北海道,NaN,634,620,594,579,569,561,547,535,534,10.5
2,50,NaN,病院数,2023000000,NaN,令和5年(2023年),3,NaN,青 森,NaN,110,109,105,102,97,94,94,90,89,7.5
3,50,NaN,病院数,2023000000,NaN,令和5年(2023年),4,NaN,岩 手,NaN,109,107,98,92,91,93,92,92,91,7.8
4,50,NaN,病院数,2023000000,NaN,令和5年(2023年),5,NaN,宮 城,NaN,149,151,146,141,142,140,136,135,135,6.0


In [39]:
# ==========================
# 2) Transform（Raw → interim）
# ==========================
# NOTE: 「令和5年(2023年)（実数）」と「令和5年(2023年)（人口10万対）」を固定抽出

df = df_raw.copy()

# prefecture
pref_col = "都道府県_005"
if pref_col not in df.columns:
    raise KeyError(f'Required column not found: "{pref_col}"')
df["prefecture"] = df[pref_col].apply(normalize_prefecture)
df = df[df["prefecture"].notna()].copy()

# years (must be provided in SUBJECT INFO)
df["fiscal_year"] = int(FISCAL_YEAR)
df["reference_year"] = int(REFERENCE_YEAR)

# raw columns (fixed)
raw_count = "令和5年(2023年)（実数）"
raw_per_100k = "令和5年(2023年)（人口10万対）"

for col in [raw_count, raw_per_100k]:
    if col not in df.columns:
        raise KeyError(f"Missing raw column: {col}")

# rename to METRIC_COLS
df = df.rename(
    columns={
        raw_count: "hospitals_count",
        raw_per_100k: "hospitals_per_100k",
    }
)

# numeric conversion (no imputation)
for col in METRIC_COLS:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .replace({"-": None, "—": None, "–": None})
    )
    df[col] = pd.to_numeric(df[col], errors="coerce")

# schema fix
df_interim = df[["prefecture", "fiscal_year", "reference_year"] + METRIC_COLS].copy()


In [40]:
# ==========================
# 3-1 Validation（保存前に必須：★CHANGEは列リストだけ）
# ==========================
print("\n=== VALIDATION ===")
num_cols = METRIC_COLS  # ★CHANGE: 数値列（基本はMETRIC_COLS）

# ★CHANGE: %列（0〜100チェック）/ 非負列（>=0チェック）
rate_cols = []     # ★CHANGE: 例 ["turnover_total", ...]
nonneg_cols = []   # ★CHANGE: 例 ["population_total", "家賃平均【円】", ...]

print("rows:", len(df_interim))

dup = df_interim.duplicated(["prefecture", "fiscal_year"]).sum() #
print("dup_keys:", dup)

check_cols = ["prefecture", "fiscal_year", "reference_year"] + num_cols
nulls = df_interim[check_cols].isna().sum()
print("nulls:\n", nulls)

# %の範囲
for c in rate_cols:
    out = ((df_interim[c] < 0) | (df_interim[c] > 100)).sum()
    print(f"{c} out_of_range:", out)

# 非負
for c in nonneg_cols:
    out = (df_interim[c] < 0).sum()
    print(f"{c} negative:", out)

# 粒度強制（47都道府県×年度1つ）
assert df_interim["fiscal_year"].nunique() == 1
assert df_interim["prefecture"].nunique() == 47

# 必須assert
assert len(df_interim) == 47
assert dup == 0
assert df_interim[check_cols].isna().sum().sum() == 0
for c in rate_cols:
    assert ((df_interim[c] < 0) | (df_interim[c] > 100)).sum() == 0
for c in nonneg_cols:
    assert (df_interim[c] < 0).sum() == 0

print("✅ validation passed")


=== VALIDATION ===
rows: 47
dup_keys: 0
nulls:
 prefecture            0
fiscal_year           0
reference_year        0
hospitals_count       0
hospitals_per_100k    0
dtype: int64
✅ validation passed


In [41]:
# ==========================
# 4-1 Save（interim）
# ==========================
df_interim.to_parquet(out_path, index=False) #ここ変更必要
print("✅ saved:", out_path)

# ==========================
# 4-2 Read-back check
# ==========================
df_check = pd.read_parquet(out_path)
print("\n=== READ-BACK ===")
print("shape:", df_check.shape)
print("columns:", df_check.columns.tolist())
display(df_check.head(3))

✅ saved: ../data/clean/interim_.parquet

=== READ-BACK ===
shape: (47, 5)
columns: ['prefecture', 'fiscal_year', 'reference_year', 'hospitals_count', 'hospitals_per_100k']


,prefecture,fiscal_year,reference_year,hospitals_count,hospitals_per_100k
0,北海道,2023,2023,534,10.5
1,青森県,2023,2023,89,7.5
2,岩手県,2023,2023,91,7.8


In [42]:
# ==========================
# 0) Config（総務省_社会生活基本調査_通勤時間_2022.xlsx）
# ==========================
SUBJECT = ""  # ★CHANGE: "rent_2024" / "jobs" / "night_shift" etc.
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/clean")

# ファイル指定（★CHANGE）
raw_path = RAW_DIR / "総務省_社会生活基本調査_通勤時間_2022.xlsx"  # ★CHANGE

# 読み込み方式（★CHANGE：csv / excel）
READ_MODE = "excel"  # ★CHANGE: "csv" or "excel"

# excelの場合（★CHANGE）
EXCEL_SHEET = None  # ★CHANGE: "第１８表ー４　有効求人倍率（実数）" など
EXCEL_HEADER = 0    # ★CHANGE: header行。未確定なら None/0 でプロファイル

# fiscal_year / reference_year（★CHANGE：ルールに従って必ず設定）
FISCAL_YEAR = 2023       # ★CHANGE: master結合キー
REFERENCE_YEAR = 2023    # ★CHANGE: 指標の観測年（年度不一致ならここをズラす）

# 保存先（★CHANGE）
out_path = PROCESSED_DIR / f"interim_{SUBJECT}.parquet"  # ★CHANGE: 命名固定したいならここだけ


In [43]:
# ==========================
# 1-1 Input Check（ファイル存在・文字コードヒント）
# Role:
#   - raw_path の存在確認
#   - 先頭バイト確認（BOM/UTF-16等の検知ヒント）
# Rules:
#   - ここでは pandas 読み込みはしない
# ==========================
assert raw_path.exists(), f"❌ ファイルが存在しません: {raw_path}"
print("target:", raw_path)

with open(raw_path, "rb") as f:
    head = f.read(16)
print("head bytes:", head)

target: ../data/raw/総務省_社会生活基本調査_通勤時間_2022.xlsx
head bytes: b'PK\x03\x04\x14\x00\x06\x00\x08\x00\x00\x00!\x00!\x8c'


In [44]:
# ==========================
# 1-2 PROFILE READ（Excelヘッダー探索用）
# 目的:
#   - ヘッダーが何行目か確認する
#   - シート構造を人間が目視する
# ルール:
#   - この段階では列加工しない
# ==========================

df_test = pd.read_excel(
    raw_path,   
    header=None               # ← ヘッダー扱いしないで全部読み込む
)

print("shape:", df_test.shape)
display(df_test.head(30))     # ← 上30行くらい見ればヘッダー位置わかる


shape: (54, 29)


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,社会生活基本調査から分かる47都道府県ランキング （令和３年社会生活基本調査結果より）,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,【1日の生活時間の使い方から 】,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,睡眠時間たっぷり！？ランキング,NaN,NaN,NaN,NaN,早起き！？ランキング,NaN,NaN,NaN,...,NaN,NaN,スローライフ！？たっぷり食事時間\nランキング,NaN,NaN,NaN,NaN,イクメン！？ランキング,NaN,NaN
3,NaN,順位,都道府県名,時間.分,NaN,NaN,順位,都道府県名,平均時刻,NaN,...,NaN,NaN,順位,都道府県名,時間.分,NaN,NaN,順位,都道府県名,時間.分
4,NaN,NaN,全国平均,7.54,NaN,NaN,NaN,全国平均,6:38,NaN,...,NaN,NaN,NaN,全国平均,1.39,NaN,NaN,NaN,全国平均,1.54
5,NaN,1,青森県,8.08,NaN,NaN,1,青森県,06:17:00,NaN,...,NaN,NaN,1,山梨県,1.45,NaN,NaN,1,奈良県,2.35
6,NaN,2,秋田県,8.06,NaN,NaN,2,岩手県,06:21:00,NaN,...,NaN,NaN,1,長野県,1.45,NaN,NaN,2,新潟県,2.33
7,NaN,3,鹿児島県,8.05,NaN,NaN,2,秋田県,06:21:00,NaN,...,NaN,NaN,3,秋田県,1.44,NaN,NaN,3,高知県,2.27
8,NaN,4,宮城県,8.04,NaN,NaN,4,長野県,06:22:00,NaN,...,NaN,NaN,4,奈良県,1.43,NaN,NaN,4,和歌山県,2.21
9,NaN,4,高知県,8.04,NaN,NaN,5,富山県,06:24:00,NaN,...,NaN,NaN,5,茨城県,1.42,NaN,NaN,5,千葉県,2.2


In [45]:
# ==========================
# 1-3 SUBJECT / METRIC定義
# ==========================

# ★CHANGE: 対象指標列（英語最終名）
METRIC_COLS = ["commute_time_min"]
print("\n=== SUBJECT INFO ===")
print("SUBJECT:", SUBJECT)
print("METRIC_COLS:", METRIC_COLS)

# ==========================
# 1-3 Read（Raw → DataFrame）
# ★CHANGE: header のみ変更（列加工・型変換はしない）
# ==========================
df_raw = pd.read_excel(raw_path, sheet_name=0, header=[2, 3])

print("\n=== RAW INFO ===")
print("shape:", df_raw.shape)
print("columns:", list(df_raw.columns))
print("dtypes:\n", df_raw.dtypes)
display(df_raw.head(5))


=== SUBJECT INFO ===
SUBJECT: 
METRIC_COLS: ['commute_time_min']

=== RAW INFO ===
shape: (50, 29)
columns: [('Unnamed: 0_level_0', 'Unnamed: 0_level_1'), ('睡眠時間たっぷり！？ランキング ', '順位'), ('睡眠時間たっぷり！？ランキング ', '都道府県名'), ('睡眠時間たっぷり！？ランキング ', '時間.分'), ('睡眠時間たっぷり！？ランキング ', '時間.分.1'), ('睡眠時間たっぷり！？ランキング ', '時間.分.2'), ('早起き！？ランキング', '順位'), ('早起き！？ランキング', '都道府県名'), ('早起き！？ランキング', '平均時刻'), ('早起き！？ランキング', '平均時刻.1'), ('早起き！？ランキング', '平均時刻.2'), ('夜更かし！？ランキング ', '順位'), ('夜更かし！？ランキング ', '都道府県名'), ('夜更かし！？ランキング ', '平均時刻'), ('夜更かし！？ランキング ', '平均時刻.1'), ('夜更かし！？ランキング ', '平均時刻.2'), ('通勤・通学時間が長い！？\nランキング ', '順位'), ('通勤・通学時間が長い！？\nランキング ', '都道府県名'), ('通勤・通学時間が長い！？\nランキング ', '時間.分'), ('通勤・通学時間が長い！？\nランキング ', '時間.分.1'), ('通勤・通学時間が長い！？\nランキング ', '時間.分.2'), ('スローライフ！？たっぷり食事時間\nランキング ', '順位'), ('スローライフ！？たっぷり食事時間\nランキング ', '都道府県名'), ('スローライフ！？たっぷり食事時間\nランキング ', '時間.分'), ('スローライフ！？たっぷり食事時間\nランキング ', '時間.分.1'), ('スローライフ！？たっぷり食事時間\nランキング ', '時間.分.2'), ('イクメン！？ランキング ', '順位'), ('イクメン！？ランキング ', '都道府県名'), ('イクメン！？ランキング ', '

Unnamed: 0_level_0 睡眠時間たっぷり！？ランキング                            早起き！？ランキング  \
  Unnamed: 0_level_1               順位 都道府県名  時間.分 時間.分.1 時間.分.2         順位   
0                NaN              NaN  全国平均  7.54    NaN    NaN        NaN   
1                NaN                1   青森県  8.08    NaN    NaN          1   
2                NaN                2   秋田県  8.06    NaN    NaN          2   
3                NaN                3  鹿児島県  8.05    NaN    NaN          2   
4                NaN                4   宮城県  8.04    NaN    NaN          4   

                          ... 通勤・通学時間が長い！？\nランキング          \
  都道府県名      平均時刻 平均時刻.1  ...               時間.分.1 時間.分.2   
0  全国平均      6:38    NaN  ...                  NaN    NaN   
1   青森県  06:17:00    NaN  ...                  NaN    NaN   
2   岩手県  06:21:00    NaN  ...                  NaN    NaN   
3   秋田県  06:21:00    NaN  ...                  NaN    NaN   
4   長野県  06:22:00    NaN  ...                  NaN    NaN   

  スローライフ！？たっぷり食事時間\nランキング                            イクメン！？ランキング               
                        順位 都道府県名  時間.分 時間.分.1 時間.分.2           順位 都道府県名  時間.分  
0                      NaN  全国平均  1.39    NaN    NaN          NaN  全国平均  1.54  
1                        1   山梨県  1.45    NaN    NaN            1   奈良県  2.35  
2                        1   長野県  1.45    NaN    NaN            2   新潟県  2.33  
3                        3   秋田県  1.44    NaN    NaN            3   高知県  2.27  
4                        4   奈良県  1.43    NaN    NaN            4  和歌山県  2.21  

[5 rows x 29 columns]

In [46]:
# ==========================
# 2) Transform（Raw → interim）
# ==========================
# NOTE: 神奈川が 1.40→64 になるのは「floatの1.4」として読まれているため。時.分を文字列化してから分変換する。

df = df_raw.copy()

group = "通勤・通学時間が長い！？\nランキング "
col_pref_raw = (group, "都道府県名")
col_time_raw = (group, "時間.分")

for c in [col_pref_raw, col_time_raw]:
    if c not in df.columns:
        raise KeyError(f"Missing raw column: {c}")

df = df[[col_pref_raw, col_time_raw]].copy()
df.columns = ["prefecture", "commute_time_min_raw"]

df["prefecture"] = df["prefecture"].apply(normalize_prefecture)
df = df[df["prefecture"].notna()].copy()

df["fiscal_year"] = int(FISCAL_YEAR)
df["reference_year"] = 2021

# ---- 時.分 → 分（floatでも「時.分」として扱う）----
def hm_to_min_str(x):
    if pd.isna(x):
        return None

    # いったん文字列へ（floatの1.4を"1.40"に戻す）
    if isinstance(x, (int, float)) and not pd.isna(x):
        # 小数第2位まで固定（分は2桁想定）
        s = f"{x:.2f}"
    else:
        s = str(x).strip()

    if s in {"", "-", "—", "–"}:
        return None

    if "." not in s:
        # "1:40" などの混入はここで落とす（想定外）
        try:
            return int(float(s))
        except:
            return None

    h, m = s.split(".", 1)

    try:
        h_i = int(h)
        m_i = int(m)
    except:
        return None

    # 分が60以上は不正（例: 1.75）なので落とす（Validationで検知）
    if not (0 <= m_i < 60):
        return None

    return h_i * 60 + m_i

df["commute_time_min"] = df["commute_time_min_raw"].apply(hm_to_min_str)
df["commute_time_min"] = pd.to_numeric(df["commute_time_min"], errors="coerce")

df_interim = df[["prefecture", "fiscal_year", "reference_year"] + METRIC_COLS].copy()


In [47]:
# ==========================
# 3-1 Validation（保存前に必須：★CHANGEは列リストだけ）
# ==========================
print("\n=== VALIDATION ===")
num_cols = METRIC_COLS  # ★CHANGE: 数値列（基本はMETRIC_COLS）

# ★CHANGE: %列（0〜100チェック）/ 非負列（>=0チェック）
rate_cols = []     # ★CHANGE: 例 ["turnover_total", ...]
nonneg_cols = []   # ★CHANGE: 例 ["population_total", "家賃平均【円】", ...]

print("rows:", len(df_interim))

dup = df_interim.duplicated(["prefecture", "fiscal_year"]).sum() #
print("dup_keys:", dup)

check_cols = ["prefecture", "fiscal_year", "reference_year"] + num_cols
nulls = df_interim[check_cols].isna().sum()
print("nulls:\n", nulls)

# %の範囲
for c in rate_cols:
    out = ((df_interim[c] < 0) | (df_interim[c] > 100)).sum()
    print(f"{c} out_of_range:", out)

# 非負
for c in nonneg_cols:
    out = (df_interim[c] < 0).sum()
    print(f"{c} negative:", out)

# 粒度強制（47都道府県×年度1つ）
assert df_interim["fiscal_year"].nunique() == 1
assert df_interim["prefecture"].nunique() == 47

# 必須assert
assert len(df_interim) == 47
assert dup == 0
assert df_interim[check_cols].isna().sum().sum() == 0
for c in rate_cols:
    assert ((df_interim[c] < 0) | (df_interim[c] > 100)).sum() == 0
for c in nonneg_cols:
    assert (df_interim[c] < 0).sum() == 0

print("✅ validation passed")


=== VALIDATION ===
rows: 47
dup_keys: 0
nulls:
 prefecture          0
fiscal_year         0
reference_year      0
commute_time_min    0
dtype: int64
✅ validation passed


In [48]:
# ==========================
# 4-1 Save（interim）
# ==========================
df_interim.to_parquet(out_path, index=False) #ここ変更必要
print("✅ saved:", out_path)

# ==========================
# 4-2 Read-back check
# ==========================
df_check = pd.read_parquet(out_path)
print("\n=== READ-BACK ===")
print("shape:", df_check.shape)
print("columns:", df_check.columns.tolist())
display(df_check.head(3))

✅ saved: ../data/clean/interim_.parquet

=== READ-BACK ===
shape: (47, 4)
columns: ['prefecture', 'fiscal_year', 'reference_year', 'commute_time_min']


,prefecture,fiscal_year,reference_year,commute_time_min
0,神奈川県,2023,2021,100
1,千葉県,2023,2021,95
2,東京都,2023,2021,95
